In [1]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
s = HTMLSession()

In [2]:
# All the categories to extract

men_clothing = ['https://www.amazon.in/s?k=men+clothing&ref=nb_sb_noss_2','men clothing']
women_clothing = ['https://www.amazon.in/s?k=women+clothing&ref=nb_sb_noss_2','women clothing']
footwear = ['https://www.amazon.in/s?k=footwear&ref=nb_sb_noss_2','footwear']
watches = ['https://www.amazon.in/s?k=watches&ref=nb_sb_noss_2','watches']
wallets = ['https://www.amazon.in/s?k=wallets&ref=nb_sb_noss_1','wallets']
bags = ['https://www.amazon.in/s?k=bags&ref=nb_sb_noss_2','bags']
jewellery = ['https://www.amazon.in/s?k=jewellery&ref=nb_sb_noss_2','jewellery']
belts = ['https://www.amazon.in/s?k=belts&ref=nb_sb_noss_1','belts']
ties = ['https://www.amazon.in/s?k=ties&ref=nb_sb_noss_1','ties']
cufflinks = ['https://www.amazon.in/s?k=cufflinks&ref=nb_sb_noss_1','cufflinks']
pocket_squares = ['https://www.amazon.in/s?k=pocket+squares&ref=nb_sb_noss_2','pocket_squares']
caps_and_hats = ['https://www.amazon.in/s?k=caps+and+hats&ref=nb_sb_noss_1','caps and hats']
mufflers_scarves_gloves = ['https://www.amazon.in/s?k=mufflers%2C+scarves+and+gloves&ref=nb_sb_noss','mufflers, scraves and gloves']
phones_cases = ['https://www.amazon.in/s?k=phone+cases&ref=nb_sb_noss_2','phones cases']
rings_and_wristwear = ['https://www.amazon.in/s?k=rings+and+wristwear&ref=nb_sb_noss','rings and wristwear']
socks = ['https://www.amazon.in/s?k=socks&ref=nb_sb_noss','socks']
bracelets = ['https://www.amazon.in/s?k=bracelets&ref=nb_sb_noss_2','bracelets']
chains = ['https://www.amazon.in/s?k=chains&ref=nb_sb_noss_1','chains']

In [3]:
# list of categories

category_list = [men_clothing,women_clothing,footwear,watches,wallets,bags,jewellery,belts,ties,cufflinks,pocket_squares,caps_and_hats,mufflers_scarves_gloves,phones_cases,rings_and_wristwear,socks,bracelets,chains]

# Functions for extraction and processing 

In [4]:
# function to extract data from the webpage

def get_data(url):
    r = s.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    return soup

# funtion to extract link for the next page in amazon.in of the category of the product

def get_nextpage(soup1):
    page1 = soup1.find('ul',{'class':'a-pagination'})
    try:
        if not page1.find('li',{'class':'a-disabled a-last'}):
            url = 'https://www.amazon.in/'+str(page1.find('li',{'class':'a-last'}).find('a')['href'])
            return url
        else:
            return False
    except:
        return 'load again'

In [5]:
# function to get sizes of the product 

def get_sizes(p_soup):
    sizes=[]
    page2 = p_soup
    size = page2.find('select',{'name':'dropdown_selected_size_name'})
    if size:
        s_list = size.find_all('option')
        for i in s_list:
            sizes.append(i.get_text(strip=True))
    else:
        sizes=[None]
    sizes.pop(0)
    if len(sizes)==0:
        sizes = None
    return sizes
    

In [6]:
# function to get secondary images list of the product 

def get_secondary_image_links(p_soup):
    page3 = p_soup
    im_list = []
    images = page3.find('ul',{'class':'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-extra-large'})
    if images and len(images.find_all('li',{'class':'a-spacing-small item'})):
        for i in images.find_all('li',{'class':'a-spacing-small item'}):
            im_list.append(i.find('img')['src'])
    else:
        im_list = [None]
    im_list.pop(0)
    if len(im_list)==0:
        im_list=None
    return im_list

In [7]:
# function to get description of the product 

def get_description(p_soup):
    page = p_soup
    desc = page.find('div',{'id':'productDescription'})
    description = []
    if desc:
        for i in desc.stripped_strings:
            description.append('->  '+i+'\n\n')
        return ''.join(description)
    else:
        return None

In [8]:
# function to extract all the needed information about all the products from amazon.in

def get_category_info(search_url):
    current_page_link = search_url[0]
    current_page = get_data(current_page_link)
    info = []
    #get all prod data 
    print("\n\n\nEXTRACTING DIFFERENT PRODUCT CATEGORY",end='')
    x=1
    while True:
        next_page=None
        #loading the page properly
        while get_nextpage(current_page)=='load again':
            current_page = get_data(current_page_link)
        if get_nextpage(current_page):
            next_page = get_data(get_nextpage(current_page))
        print(".....extracting "+str(x)+"th page",end='')
        p = current_page.find_all('div',{'data-component-type':'s-search-result'})
        for l in p:
            
            # extracting gender using name of the product
            
            gender = None
            product_name = l.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'})
            if product_name:
                product_name = str(product_name.get_text())
                if ' men' in search_url[1].lower()+product_name.lower() and 'women' in search_url[1].lower()+product_name.lower():
                    gender = 'any'
                elif 'women' in search_url[1].lower()+product_name.lower():
                    gender = 'female'
                elif ' men' in search_url[1].lower()+product_name.lower():
                    gender = 'male'
                else:
                    gender = 'any'

            else:
                product_name = None
                
                            
            # extracting brand name 
            
            brand = l.find('span',{'class':'a-size-base-plus a-color-base'})
            if brand:
                brand = str(brand.get_text())
            else:
                brand = None
                
            # extracting price 
            
            price = l.find('span',{'class':'a-price-whole'})
            if price:
                if ',' in price.get_text():
                    price = float(''.join(price.get_text().split(',')))
                else:
                    price = float(price.get_text())
            else:
                price = 500
           
                
            # extracting MRP 
            MRP = l.find('span',{'class':'a-price a-text-price'})
            if MRP:
                MRP=MRP.find('span',{'aria-hidden':'true'}).get_text()[1:] 
                if ',' in MRP:
                    MRP=float(''.join(MRP.split(',')))
                else:
                    MRP = float(MRP)
            else:
                MRP = 1000
                
            product_link = 'https://www.amazon.in'+str(l.find('a',{'class':'a-link-normal s-no-outline'})['href'])
            product_soup = get_data(product_link)
            
            # collecting all data
            d = {
                'website':'https://www.amazon.in',
                'product_name':product_name,
                'product_brand':brand,
                'product_category':search_url[1],
                'sizes_available': str(get_sizes(product_soup))[1:-1],
                'price':price,
                'MRP':MRP,
                'gender':gender,
                'description': get_description(product_soup),
                'primary_image_link':l.find('img',{'class':'s-image'})['src'],
                'secondary_image_links':str(get_secondary_image_links(product_soup))[1:-1],
                'product_link': product_link
            }
            info.append(d)
        x+=1
        if next_page:
            current_page_link = get_nextpage(current_page)
            current_page = next_page
        else:
            break
    return info

# Extracting whole data and Building the Dataframe 

In [ ]:
import pandas as pd

df1 = pd.DataFrame()
for item in category_list:
    j=get_category_info(item)
    df2 = pd.DataFrame(j)
    df1 = df1.append(df2, ignore_index = True)


extracting different product category.....extracting 1th page.....extracting 2th page.....extracting 3th page.....extracting 4th page.....extracting 5th page.....extracting 6th page.....extracting 7th page
extracting different product category.....extracting 1th page.....extracting 2th page.....extracting 3th page.....extracting 4th page.....extracting 5th page.....extracting 6th page.....extracting 7th page
extracting different product category.....extracting 1th page.....extracting 2th page.....extracting 3th page.....extracting 4th page.....extracting 5th page.....extracting 6th page.....extracting 7th page
extracting different product category.....extracting 1th page.....extracting 2th page.....extracting 3th page.....extracting 4th page.....extracting 5th page.....extracting 6th page.....extracting 7th page
extracting different product category.....extracting 1th page.....extracting 2th page.....extracting 3th page.....extracting 4th page.....extracting 5th page.....extracting 6t

# Storing dataframe in sqLite3 database

In [ ]:
import sqlite3

engine = sqlite3.connect('AllData.db')
c = engine.cursor()

try:
    c.execute('CREATE TABLE PRODUCTS (website,product_name,product_brand,product_category,sizes_available,price,MRP,gender,description,primary_image_link,secondary_image_links,product_link)')
    engine.commit()
    
except Exception as e:
    if str(e)=='table PRODUCTS already exists' and str(e.__class__) == "<class 'sqlite3.OperationalError'>":
        c.execute('DROP TABLE PRODUCTS')
        engine.commit()

        df1.to_sql('PRODUCTS',engine, if_exists='replace', index = False)
    else:
        print(e.__class__,e)
else:
    df1.to_sql('PRODUCTS',engine, if_exists='replace', index = False)



## Closing the database and showing data stored

In [ ]:
from contextlib import closing

with closing(sqlite3.connect("AllData.db")) as engine:
    with closing(engine.cursor()) as c:
        c.execute('''SELECT * FROM PRODUCTS''')
        for row in c.fetchall():
            print(row)



